In [44]:
import pickle

with open('Data/all_game_detail_2024.pkl', 'rb') as f:
    existing_data = pickle.load(f)

In [50]:
len(existing_data[:206])

205

In [25]:
import pandas as pd

data_path = 'Data and Scripts/Data/all_players.csv'
data_dict = 'fbref_dictionary.csv'
players = pd.read_csv(data_path)
dataDict = pd.read_csv(data_dict)

In [2]:
import pandas as pd

odds = pd.read_csv('Data And Scripts/Data/historical_odds_goals.csv')

In [114]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd

def get_header_mapping(url):
    """
    Creates a mapping table between aria-labels and data-tips from HTML table headers from a URL
    
    Args:
        url (str): URL to scrape
        
    Returns:
        DataFrame: Mapping of header information including field names and descriptions
    """
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find all table headers with aria-labels
    headers = soup.find_all('th')
    
    # Create mapping lists
    mapping_data = []
    seen_pairs = set()  # Track unique combinations
    
    for header in headers:
        aria_label = header.get('aria-label', '')
        data_tip = header.get('data-tip', '')
        data_stat = header.get('data-stat', '')
        data_over_header = header.get('data-over-header', '')
        header_text = header.get_text(strip=True)
        
        # Create unique identifier
        unique_pair = (aria_label, data_over_header, header_text)
        
        # Skip if we've seen this exact combination
        if unique_pair in seen_pairs or not (aria_label or data_over_header or header_text):
            continue
            
        seen_pairs.add(unique_pair)
            
        # Parse data-tip HTML and split on <br> tags
        if data_tip:
            tip_soup = BeautifulSoup(data_tip, 'html.parser')
            tip_lines = [line.strip() for line in tip_soup.get_text(separator='\n').split('\n') if line.strip()]
        else:
            tip_lines = []
        
        # Construct field name
        field_prefix = f"{data_over_header} - " if data_over_header else ""
        field_name = f"{field_prefix}{header_text}"
        try:
            description = ' '.join(tip_lines[1:])
        except:
            description = ''
        # Add to mapping
        mapping_data.append({
            'field': field_name,
            'labelDescription': aria_label,
            'subCategory': data_over_header,
            'columnText': header_text,
            'detailDescription': description,
            'stat': data_stat
        })
    
    # Convert to DataFrame and filter out rows with empty labelDescription
    df = pd.DataFrame(mapping_data)
    df = df[df['labelDescription'] != ''].reset_index(drop=True)
    
    return df


# Example usage:
url = "https://fbref.com/en/matches/cc5b4244/Manchester-United-Fulham-August-16-2024-Premier-League"
mapping_df = get_header_mapping(url)
display(mapping_df)

,field,labelDescription,subCategory,columnText,detailDescription,stat
0,Player,Player,,Player,,player
1,#,Shirt Number,,#,Shirt Number,shirtnumber
2,Nation,Nation,,Nation,"First, we check our records in international p...",nationality
3,Pos,Position,,Pos,Position most commonly played by the player GK...,position
4,Age,Age,,Age,Age at season start Given on August 1 for wint...,age
...,...,...,...,...,...,...
141,Notes,Notes,,Notes,,notes
142,SCA 1 - Player,Player,SCA 1,Player,,sca_1_player
143,SCA 1 - Event,Event,SCA 1,Event,,sca_1_type
144,SCA 2 - Player,Player,SCA 2,Player,,sca_2_player


In [151]:
cutoff_idx = mapping_df[mapping_df['field'] == 'Sweeper - AvgDist'].index[0] + 1
players_map = mapping_df.iloc[:cutoff_idx].copy()

# Add new rows with just field populated
new_rows = pd.DataFrame({
    'field': ['game', 'Season', 'team'],
    'labelDescription': ['Teams in the Game', 'Premier League Season', 'Players Team'],
    'subCategory': ['', '', ''],
    'columnText': ['', '', ''],
    'detailDescription': ['', '', ''],
    'stat': ['', '', '']
})

players_map = pd.concat([
    players_map.iloc[:6],  # First row
    new_rows,              # New rows
    players_map.iloc[6:]   # Rest of the rows
], ignore_index=True)

# Create the example values dictionary first
example_values_dict = {}
for field in players_map['field']:
    if field in players.columns:
        example_values_dict[field] = players[field].unique().tolist()[:5]
    else:
        example_values_dict[field] = []  # or some default value

# Then apply the mapping
players_map['exampleValues'] = players_map['field'].map(example_values_dict)
players_map.to_csv('mappings/players_map.csv', index = False)

players_map.iloc[:10]

,field,labelDescription,subCategory,columnText,detailDescription,stat,exampleValues
0,Player,Player,,Player,,player,"[Marcus Rashford, Romelu Lukaku, Alexis Sánche..."
1,#,Shirt Number,,#,Shirt Number,shirtnumber,"[10.0, 9.0, 7.0, 8.0, 6.0]"
2,Nation,Nation,,Nation,"First, we check our records in international p...",nationality,"[eng ENG, be BEL, cl CHI, es ESP, fr FRA]"
3,Pos,Position,,Pos,Position most commonly played by the player GK...,position,"[FW, LW, RW, LM, CM]"
4,Age,Age,,Age,Age at season start Given on August 1 for wint...,age,"[20-283, 25-089, 29-234, 30-104, 25-148]"
5,Min,Minutes,,Min,,minutes,"[66, 24, 90, 83, 7]"
6,game,Teams in the Game,,,,,"[Manchester Utd vs Leicester City, Newcastle U..."
7,Season,Premier League Season,,,,,"[2018, 2019, 2020, 2021, 2022]"
8,team,Players Team,,,,,"[Manchester Utd, Leicester City, Newcastle Utd..."
9,Performance - Gls,Goals,Performance,Gls,Goals scored or allowed,goals,"[0, 1, 2, 3, 4]"


In [119]:
fixtures_path = 'Data and Scripts/Data/fixtures.csv'
fixtures = pd.read_csv(fixtures_path)

In [157]:
fixtures[['homeGoals', 'awayGoals']] = fixtures['Score'].str.split('–', expand=True)
home = fixtures[['Wk', 'Date', 'Home', 'Game', 'Season', 'homeGoals']].rename(columns={'Home': 'Team', 'homeGoals': 'teamGoals'})
away = fixtures[['Wk', 'Date', 'Away', 'Game', 'Season', 'awayGoals']].rename(columns={'Away': 'Team', 'awayGoals': 'teamGoals'})

gameGoals = pd.concat([home, away], ignore_index=True)
gameGoals = gameGoals[gameGoals['teamGoals'].notna()]

gameGoals['teamGoals'] = gameGoals['teamGoals'].astype(int)
gameGoals['Season'] = gameGoals['Season'].astype(int)

# Create example values dictionary first
example_values_dict = {
    col: gameGoals[col].unique().tolist()[:5] 
    for col in gameGoals.columns
}

# Then create the dictionary DataFrame
gameGoalsDict = pd.DataFrame({'field': gameGoals.columns})
gameGoalsDict['exampleValues'] = gameGoalsDict['field'].map(example_values_dict)
gameGoals.to_csv('mappings/gameGoals.csv', index = False)



In [156]:
players.iloc[:1000].to_csv('all_players.csv')

In [17]:
# utils.py
import pandas as pd
import os
import logging
from betfair_api import BetfairAPI
import streamlit as st

def initialize_betfair_api():
    """Initialize Betfair API with credentials"""
    return BetfairAPI(
        username=st.secrets["betfair_username"],
        password=st.secrets["betfair_password"],
        app_key=st.secrets["betfair_app_key"],
        cert_path=st.secrets["betfair_cert_path"],
        key_path=st.secrets["betfair_key_path"]
    )

api = initialize_betfair_api()

DATA_DIR = os.path.join("Data And Scripts", "Data")
odds_df = api.get_odds(market_type='goals')

filename = f'historical_odds_goals.csv'
filepath = os.path.join(DATA_DIR, filename)

# Ensure we have a clean DataFrame with consistent columns
required_columns = ['eventName', 'marketType', 'date', 'under', 'over', 'timestamp']
current_odds = odds_df[required_columns].copy()

# Add timestamp if missing
if 'timestamp' not in current_odds.columns:
    current_odds['timestamp'] = pd.Timestamp.now()

if os.path.exists(filepath):
    # Load historical data
    historical_df = pd.read_csv(filepath)
    historical_df['timestamp'] = pd.to_datetime(historical_df['timestamp'])
    
    # Ensure historical data has same columns
    historical_df = historical_df[required_columns]
    
    # Remove old data (keep last 30 mins)
    cutoff_time = pd.Timestamp.now() - pd.Timedelta(minutes=30)
    historical_df = historical_df[historical_df['timestamp'] > cutoff_time]
    
    # Convert both DataFrames to simple structures
    historical_simple = pd.DataFrame(historical_df.values, columns=historical_df.columns)
    current_simple = pd.DataFrame(current_odds.values, columns=current_odds.columns)

    # Now try the concatenation
    combined_df = pd.concat([historical_simple, current_simple], axis=0, ignore_index=True)

    
    # Remove duplicates keeping latest timestamp
    combined_df = (combined_df.sort_values('timestamp')
                    .drop_duplicates(subset=['eventName', 'marketType', 'date', 'under', 'over'], 
                                keep='last')
                    .reset_index(drop=True))
else:
    combined_df = current_odds

# Convert numeric columns
for col in ['over', 'under']:
    combined_df[col] = pd.to_numeric(combined_df[col], errors='coerce')

# Sort and save
combined_df = combined_df.sort_values(['eventName', 'marketType', 'timestamp'])
combined_df.to_csv(filepath, index=False)
        



2025-01-19 15:18:17,013 - INFO - Successfully logged in to Betfair


In [18]:
combined_df

,eventName,marketType,date,under,over,timestamp
34,Bournemouth vs Nott'ham Forest,OVER_UNDER_05,2025-01-25T15:00:00.000Z,1.52,1.07,2025-01-19 15:18:18.384616
35,Bournemouth vs Nott'ham Forest,OVER_UNDER_25,2025-01-25T15:00:00.000Z,1.52,1.21,2025-01-19 15:18:18.384616
1,Brighton vs Everton,OVER_UNDER_05,2025-01-25T15:00:00.000Z,1.30,1.06,2025-01-19 15:18:18.384616
2,Brighton vs Everton,OVER_UNDER_25,2025-01-25T15:00:00.000Z,1.30,1.90,2025-01-19 15:18:18.384616
19,Chelsea vs Wolves,OVER_UNDER_05,2025-01-20T20:00:00.000Z,27.00,1.02,2025-01-19 15:18:18.384616
15,Chelsea vs Wolves,OVER_UNDER_15,2025-01-20T20:00:00.000Z,7.60,1.13,2025-01-19 15:18:18.384616
12,Chelsea vs Wolves,OVER_UNDER_25,2025-01-20T20:00:00.000Z,3.25,1.43,2025-01-19 15:18:18.384616
16,Chelsea vs Wolves,OVER_UNDER_35,2025-01-20T20:00:00.000Z,1.91,2.08,2025-01-19 15:18:18.384616
13,Chelsea vs Wolves,OVER_UNDER_45,2025-01-20T20:00:00.000Z,1.38,3.50,2025-01-19 15:18:18.384616
14,Chelsea vs Wolves,OVER_UNDER_55,2025-01-20T20:00:00.000Z,1.16,6.40,2025-01-19 15:18:18.384616


In [14]:
current_odds.head(7)

,eventName,marketType,date,under,over,timestamp
0,Manchester Utd vs Brighton,OVER_UNDER_75,2025-01-19T14:00:00.000Z,NaN,5.30,2025-01-19 15:15:11.205691
1,Manchester Utd vs Brighton,OVER_UNDER_85,2025-01-19T14:00:00.000Z,NaN,240.00,2025-01-19 15:15:11.205691
2,Manchester Utd vs Brighton,OVER_UNDER_65,2025-01-19T14:00:00.000Z,1.03,20.00,2025-01-19 15:15:11.205691
3,Manchester Utd vs Brighton,OVER_UNDER_25,2025-01-19T14:00:00.000Z,4.00,1.01,2025-01-19 15:15:11.205691
4,Manchester Utd vs Brighton,OVER_UNDER_45,2025-01-19T14:00:00.000Z,1.71,1.37,2025-01-19 15:15:11.205691
5,Manchester Utd vs Brighton,OVER_UNDER_55,2025-01-19T14:00:00.000Z,1.17,6.20,2025-01-19 15:15:11.205691
6,Manchester Utd vs Brighton,OVER_UNDER_35,2025-01-19T14:00:00.000Z,1.65,1.01,2025-01-19 15:15:11.205691


In [2]:
from src.utils.historical_betfair import BetfairHistorical
import datetime
import pandas as pd

fixtures_path = '/Users/danielcrake/Desktop/Football Betting 2025/Data and Scripts/Data/fixtures.csv'
fixtures = pd.read_csv(fixtures_path)
player_stats = pd.read_csv('/Users/danielcrake/Desktop/Football Betting 2025/Data And Scripts/Data/all_players.csv')


fixtures[['homeGoals', 'awayGoals']] = fixtures['Score'].str.split('–', expand=True)
home = fixtures[['Wk', 'Date', 'Home', 'Game', 'Season', 'homeGoals']].rename(columns={'Home': 'Team', 'homeGoals': 'teamGoals'})
away = fixtures[['Wk', 'Date', 'Away', 'Game', 'Season', 'awayGoals']].rename(columns={'Away': 'Team', 'awayGoals': 'teamGoals'})

gameGoals = pd.concat([home, away], ignore_index=True)
gameGoals = gameGoals[gameGoals['teamGoals'].notna()]

gameGoals['teamGoals'] = gameGoals['teamGoals'].astype(int)
gameGoals['Season'] = gameGoals['Season'].astype(int)
gameGoals['Date'] = pd.to_datetime(gameGoals['Date'])

# Example date range
start_date = datetime.date(2024, 12, 10)
end_date = datetime.date(2024, 12, 20)
games = list(gameGoals[(gameGoals['Date'] >= pd.to_datetime(start_date)) & (gameGoals['Date'] <= pd.to_datetime(end_date))].Game.unique())

api = BetfairHistorical(games=games)

historical_odds = api.get_historical_odds(start_date, end_date)
print(f"Retrieved {len(historical_odds)} historical odds records")

# Clean the odds if needed (with specified minutes before kickoff)
cleaned_odds = api.odds_clean(historical_odds, mins_before_ko=120, price_strategy='max')

2025-02-05 22:04:33,633 - INFO - Requesting historical data from 2024-12-10 to 2024-12-20
2025-02-05 22:04:34,708 - INFO - Retrieved file list with 1314 files
2025-02-05 22:04:34,759 - INFO - Downloading 243 new files
2025-02-05 22:04:37,926 - ERROR - Error verifying bz2 file Data/Price Data/1.237206996.bz2: Invalid data stream
2025-02-05 22:04:37,930 - WARNING - Downloaded file /xds_nfs/edp_processed/BASIC/2024/Dec/24/33867955/1.237206996.bz2 is corrupt (attempt 1/3)
2025-02-05 22:04:38,129 - ERROR - Error verifying bz2 file Data/Price Data/1.237209296.bz2: Invalid data stream
2025-02-05 22:04:38,152 - WARNING - Downloaded file /xds_nfs/edp_processed/BASIC/2024/Dec/20/33868143/1.237209296.bz2 is corrupt (attempt 1/3)
2025-02-05 22:04:38,156 - ERROR - Error verifying bz2 file Data/Price Data/1.237209293.bz2: Invalid data stream
2025-02-05 22:04:38,164 - WARNING - Downloaded file /xds_nfs/edp_processed/BASIC/2024/Dec/20/33868143/1.237209293.bz2 is corrupt (attempt 1/3)
2025-02-05 22:04:

Retrieved 17820 historical odds records


In [5]:
cleaned_odds

,eventName,marketType,under,publishTime_x,over,publishTime_y,name
0,Arsenal vs Everton,OVER_UNDER_15,5.80,2024-12-14 13:01:33.349000+00:00,1.25,2024-12-14 13:01:33.349000+00:00,Under 1.5 Goals
1,Bournemouth vs West Ham,OVER_UNDER_15,6.60,2024-12-16 18:04:33.469000+00:00,1.22,2024-12-16 18:04:33.469000+00:00,Over 1.5 Goals
2,Brighton vs Crystal Palace,OVER_UNDER_15,4.90,2024-12-15 12:06:31.892000+00:00,1.30,2024-12-15 12:06:31.892000+00:00,Over 1.5 Goals
3,Chelsea vs Brentford,OVER_UNDER_15,9.80,2024-12-15 17:05:33.324000+00:00,1.14,2024-12-15 17:05:33.324000+00:00,Over 1.5 Goals
4,Liverpool vs Fulham,OVER_UNDER_15,8.60,2024-12-14 13:50:33.645000+00:00,1.15,2024-12-14 13:50:33.645000+00:00,Over 1.5 Goals
5,Manchester City vs Manchester Utd,OVER_UNDER_15,7.40,2024-12-15 14:32:36.307000+00:00,1.17,2024-12-15 14:32:36.307000+00:00,Over 1.5 Goals
6,Newcastle Utd Utd vs Leicester City City,OVER_UNDER_15,8.80,2024-12-14 13:02:29.561000+00:00,1.14,2024-12-14 13:02:29.561000+00:00,Over 1.5 Goals
7,Nott'ham Forest vs Aston Villa,OVER_UNDER_15,3.85,2024-12-14 15:30:34.139000+00:00,1.43,2024-12-14 15:30:34.139000+00:00,Over 1.5 Goals
8,Southampton vs Tottenham,OVER_UNDER_15,12.00,2024-12-15 17:08:56.234000+00:00,1.11,2024-12-15 17:08:56.234000+00:00,Over 1.5 Goals
9,Wolves vs Ipswich Town Town,OVER_UNDER_15,4.70,2024-12-14 13:02:31.332000+00:00,1.30,2024-12-14 13:02:31.332000+00:00,Over 1.5 Goals


In [15]:
pd.concat([historical_df, current_odds], axis=0, ignore_index=True)

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 50 and the array at index 1 has size 49

In [11]:
print("Historical DataFrame info:")
print(historical_df.dtypes)
print(historical_df.shape)

print("\nCurrent odds info:")
print(current_odds.dtypes)
print(current_odds.shape)

Historical DataFrame info:
eventName             object
marketType            object
date                  object
under                float64
over                 float64
timestamp     datetime64[ns]
dtype: object
(50, 6)

Current odds info:
eventName             object
marketType            object
date                  object
under                float64
over                 float64
timestamp     datetime64[us]
dtype: object
(49, 6)
